In [1]:


VIDEO_SOURCE="IMG_6292.MOV"

import os
import dotenv
# import required libraries
from vidgear.gears import CamGear
from vidgear.gears import WriteGear
import cv2
import logging
logging.basicConfig(level=logging.INFO, format="[%(asctime)s] [%(levelname)s] %(message)s")

dotenv.load_dotenv()
YOUTUBE_STREAM_KEY = os.getenv("YOUTUBE_STREAM_KEY")
print("Got %s stream key" % YOUTUBE_STREAM_KEY)

# define video source

# Open stream
stream = CamGear(source=VIDEO_SOURCE, logging=True).start()

# define required FFmpeg optimizing parameters for your writer
# [NOTE]: Added VIDEO_SOURCE as audio-source
# [WARNING]: VIDEO_SOURCE must contain audio
output_params = {
    "-i": VIDEO_SOURCE,
    "-acodec": "aac",
    "-ar": 44100,
    "-b:a": 712000,
    "-vcodec": "libx264",
    "-preset": "medium",
    "-b:v": "4500k",
    "-bufsize": "512k",
    "-pix_fmt": "yuv420p",
    "-f": "flv",
}



# Define writer with defined parameters and
writer = WriteGear(
    output="rtmp://a.rtmp.youtube.com/live2/{}".format(YOUTUBE_STREAM_KEY),
    logging=True,
    **output_params
)

# loop over
while True:

    # read frames from stream
    frame = stream.read()

    # check for frame if Nonetype
    if frame is None:
        break

    # {do something with the frame here}

    # write frame to writer
    writer.write(frame)

# safely close video stream
stream.stop()

# safely close writer
writer.close()

22:10:27 ::    Helper     ::   INFO   :: Running VidGear Version: 0.3.3
22:10:27 ::    CamGear    ::  DEBUG   :: Enabling Threaded Queue Mode for the current video source!


AssertionError: [WriteGear:ERROR] :: The `output_filename` parameter has been renamed to `output`. Refer Docs for more info.